# Along-slope velocity

Calculate the velocity component tangent to the a bathymetry contour.

Load modules

In [1]:
from pathlib import Path
import intake
import cosima_cookbook as cc
from dask.distributed import Client
import numpy as np
import xarray as xr

import xgcm
import cf_xarray

# For plotting
import cartopy.crs as ccrs
import matplotlib.pyplot as plts
import matplotlib.path as mpath
import cmocean as cm
import pyproj

By default retain metadata after operations. This can retain out of date metadata, so some caution is required

In [2]:
xr.set_options(keep_attrs=True)

Start a cluster with multiple cores

In [3]:
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34393,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 7
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:34583,Total threads: 1
Dashboard: /proxy/36027/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:36069,


In [4]:
session = cc.database.create_session()

Open the catalogue & define experiment

In [5]:
EXPERIMENT = '01deg_jra55v13_ryf9091'

In [6]:
catalog = intake.cat.access_nri
cat_filtered = catalog.search(name=EXPERIMENT)
cat_filtered

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{seaIce, ocean}","{1day, 3hr, 3mon, 1mon, fx}","{temp_vdiffuse_impl, salt, drag_coeff, lprec, total_ocean_swflx_vis, aicen_m, total_ocean_evap, alvdf_ai_m, temp_xflux_adv, sea_level, u, total_ocean_swflx, ULON, evap, vhrho_nt, sst_m, hs_m, vert..."


Limit to Southern Ocean and single RYF year

In [7]:
LAT_SLICE  = slice(-80, -59)
START_TIME = '2086-01-01'
END_TIME   = '2086-12-31'


In [8]:
esm_datastore = cat_filtered.to_source()
esm_datastore

,unique
path,11947
realm,2
variable,178
frequency,5
start_date,3361
end_date,3361
variable_long_name,181
variable_standard_name,36
variable_cell_methods,3
variable_units,50


In [9]:
# hu:  'hu': 'ocean depth on u-cells',
hu_vars = esm_datastore.search(variable='hu')
"""
CC version uses `n=1` to grab first result from the query. This appears to be a trick to avoid 
getting this error:
    `ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation`
from xarray when calling `esm_datastore.to_dataset_dict()`.

@CT: current belief is that this is something to do with coordinate variables not having dimension coordinates or 
something - we'll figure that out as we continue to delve into things.
"""


HU_VAR_PATH = hu_vars.df['path'][0]
hu_vars = hu_vars.search(path=HU_VAR_PATH)


hu = hu_vars.to_dataset_dict(progress_bar=False).get('ocean_grid.fx').drop(
    ['geolat_c','geolon_c']
).sel(
    yu_ocean=LAT_SLICE
).load()['hu']



--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id.frequency'


Load bathymetry data. Discard the geolon and geolat coordinates: these are 2D curvilinear coordinates that are only required when working above 65N

Load velocity data, limit to upper 500m and take the mean in time

In [10]:
# hu:  'hu': 'ocean depth on u-cells',
u_vars, v_vars = (esm_datastore.search(variable='u',filename='ocean.nc'),
                  esm_datastore.search(variable='v',filename='ocean.nc'))

u_intake_dict, v_intake_dict = (u_vars.to_dataset_dict(progress_bar=False),
                                v_vars.to_dataset_dict(progress_bar=False))

# .get('ocean_grid.fx').drop(
#     ['geolat_c','geolon_c']
# ).sel(
#     yu_ocean=LAT_SLICE
# ).load()['u']

u, v = u_intake_dict.get('ocean.3mon'), v_intake_dict.get('ocean.1mon')
u, v = (u.sel(yu_ocean=LAT_SLICE).sel(st_ocean=slice(0,500)).mean('time')['u'],
        v.sel(yu_ocean=LAT_SLICE).sel(st_ocean=slice(0,500)).mean('time')['v'])


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id.frequency'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id.frequency'


Load model grid information directly from a grid data file

In [11]:
grid_data_path = Path('/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/')
grid = xr.open_mfdataset(grid_data_path / 'ocean_grid.nc', combine='by_coords').drop(['geolon_t', 'geolat_t', 'geolon_c', 'geolat_c'])

  grid = xr.open_mfdataset(grid_data_path / 'ocean_grid.nc', combine='by_coords').drop(['geolon_t', 'geolat_t', 'geolon_c', 'geolat_c'])



### Along-slope velocity

We calculate the along-slope velocity component by projecting the velocity field to the tangent vector, $u_{along} = \boldsymbol{u \cdot \hat{t}}$, and the cross-slope component by projecting to the normal vector, $v_{cross} = \boldsymbol{u \cdot \hat{n}}$. The schematic below defines the unit normal normal and tangent vectors for a given bathymetric contour, $\boldsymbol{n}$ and $\boldsymbol{t}$ respectively. 

![Sketch of topographic gradient](images/topographic_gradient_sketch.png)

Accordingly, the code below calculates the along-slope velocity component as

$$ u_{along} = (u,v) \boldsymbol{\cdot} \left(\frac{h_y}{|\nabla h|} , -\frac{h_x}{|\nabla h|}\right) = 
u \frac{h_y}{|\nabla h|} - v \frac{h_x}{|\nabla h|}, $$  

and similarly the cross-slope velocity component as

$$ v_{cross} = (u,v) \boldsymbol{\cdot} \left(\frac{h_x}{|\nabla h|} , \frac{h_y}{|\nabla h|}\right)  = 
u \frac{h_x}{|\nabla h|} + v \frac{h_y}{|\nabla h|}.$$ 


We need the derivatives of the bathymetry which we compute using the `xgcm` functionality.

In [12]:
# Give information on the grid: location of u (momentum) and t (tracer) points on B-grid 
ds = xr.merge([hu, grid])
ds.coords['xt_ocean'].attrs.update(axis='X')
ds.coords['xu_ocean'].attrs.update(axis='X', c_grid_axis_shift=0.5)
ds.coords['yt_ocean'].attrs.update(axis='Y')
ds.coords['yu_ocean'].attrs.update(axis='Y', c_grid_axis_shift=0.5)

grid = xgcm.Grid(ds, periodic=['X'])

# Take topographic gradient (simple gradient over one grid cell) and move back to u-grid
dhu_dx = grid.interp( grid.diff(ds.hu, 'X') / grid.interp(ds.dxu, 'X'), 'X')

# In meridional direction, we need to specify what happens at the boundary
dhu_dy = grid.interp( grid.diff(ds.hu, 'Y', boundary='extend') / grid.interp(ds.dyt, 'X'), 'Y', boundary='extend')

# Select latitude slice
dhu_dx = dhu_dx.sel(yu_ocean=LAT_SLICE)
dhu_dy = dhu_dy.sel(yu_ocean=LAT_SLICE)

# Magnitude of the topographic slope (to normalise the topographic gradient)
topographic_slope_magnitude = np.sqrt(dhu_dx**2 + dhu_dy**2)

This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg

  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg



Calculate along-slope velocity component

In [ ]:
# Along-slope velocity
alongslope_velocity = u * dhu_dy / topographic_slope_magnitude - v * dhu_dx / topographic_slope_magnitude

# Load the data
alongslope_velocity = alongslope_velocity.load()
# warnings might come up in points where we divide by NaN/0,
# i.e., when there is no topographic gradient and warning can be ignored

This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(



Vertical averaging (we only need this to plot the velocity on a map)

In [ ]:
# Import edges of st_ocean and add lat/lon dimensions:
st_edges_args = {
    "expt" : EXPERIMENT,
    "variable" : 'st_edges_ocean',
    "n" : 1,
    "start_time" : START_TIME,
    "end_time" : END_TIME,
}
st_edges_ocean = cc.querying.getvar(
    session=session,
    **st_edges_args
)
st_edges_ocean

In [ ]:
"""
`st_edges_ocean` isn't in the catalogue - this appears to be because it's a coordinate and 
not a variable.

Not sure if there's a way to access it using intake - for now, lets try to grab it using a direct file open
"""

In [ ]:
salt_ds = esm_datastore.search(path='/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/*',variable='salt',frequency='1mon').to_dataset_dict()
salt_ds['ocean.1mon']

In [ ]:
st_edges_data_path = Path('/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/ocean.nc')
st_edges_ds = xr.open_dataset(st_edges_data_path)
st_edges_ocean = st_edges_ds.coords['st_edges_ocean']

In [ ]:
st_edges_array = st_edges_ocean.expand_dims({'yu_ocean': u.yu_ocean, 'xu_ocean': u.xu_ocean}, axis=[1, 2])

In [ ]:
# Adjust edges at bottom for partial thickness:
st_edges_with_partial = st_edges_array.where(st_edges_array<hu, other=hu)
thickness = st_edges_with_partial.diff(dim='st_edges_ocean')

# Change coordinate of thickness to st_ocean (needed for multipling with other variables):
st_ocean_args = {
    "expt" : EXPERIMENT,
    "variable" : 'st_ocean',
    "n" : 1,
}
st_ocean = cc.querying.getvar(
    session=session,
    **st_ocean_args
)
thickness['st_edges_ocean'] = st_ocean.values
thickness = thickness.rename(({'st_edges_ocean': 'st_ocean'}))
thickness = thickness.sel(st_ocean=slice(0, 500))

# Depth average gives us the barotropic velocity
barotropic_alongslope_velocity = (alongslope_velocity * thickness).sum('st_ocean') / thickness.sum('st_ocean')

### Plotting

Create a circular path to clip plots

In [ ]:
theta  = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.45
verts  = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

Create a land mask for plotting, set land cells to 1 and rest to NaN

In [ ]:
land = xr.where(np.isnan(hu.rename('land')), 1, np.nan)

#### Map of along-slope velocity with bathymetry contours

In [ ]:
fig = plt.figure(1, figsize=(15, 15))

ax = plt.subplot(1, 1, 1, projection=ccrs.SouthPolarStereo(), facecolor="darkgrey")
ax.set_boundary(circle, transform=ax.transAxes)
    
# Filled land 
land.plot.contourf(ax=ax, colors='darkgrey', zorder=2,
                   transform=ccrs.PlateCarree(), add_colorbar=False)

# Coastline
land.fillna(0).plot.contour(ax=ax, colors='k', levels=[0, 1],
                            transform=ccrs.PlateCarree(), add_colorbar=False)

# Depth contours
hu.plot.contour(ax=ax, levels=[500, 1000, 2000, 3000],
                colors='0.2', linewidths=[0.5, 2, 0.5, 0.5], alpha=0.5,
                transform=ccrs.PlateCarree())

# Along slope barotropic velocity
sc = barotropic_alongslope_velocity.plot(ax = ax, cmap=cm.cm.curl,
                                         transform=ccrs.PlateCarree(), vmin=-0.3, vmax=0.3,
                                         cbar_kwargs={'orientation': 'vertical',
                                                      'shrink': 0.25,
                                                      'extend': 'both',
                                                      'label': None,
                                                      'aspect': 8})
  
ax.set_title('Along-slope barotropic velocity (m s$^{-1}$)');

In [ ]:
client.close()